# Tool Permissions

Tool permissions define authority boundaries for what an agent can observe or mutate. This notebook demonstrates two approaches: filtering tools before agent construction, and enforcing permissions at runtime.

In [1]:
from agentic_patterns.core.agents import get_agent, run_agent
from agentic_patterns.core.tools import (
    ToolPermission,
    tool_permission,
    get_permissions,
    filter_tools_by_permission,
    enforce_tools_permissions,
)

## Define Tools

Tools are annotated with required permissions using the `@tool_permission` decorator. Without the decorator, tools default to READ permission.

In [2]:
@tool_permission(ToolPermission.READ)
def get_balance(account_id: str) -> float:
    """Get the current balance of an account."""
    print(f"Reading balance for account: {account_id}")
    return 1500.00


@tool_permission(ToolPermission.READ)
def get_transactions(account_id: str, limit: int = 10) -> list[dict]:
    """Get recent transactions for an account."""
    print(f"Reading transactions for account: {account_id}")
    return [{"id": "tx1", "amount": -50.00, "description": "Coffee shop"}]


@tool_permission(ToolPermission.WRITE)
def transfer_funds(from_account: str, to_account: str, amount: float) -> bool:
    """Transfer funds between accounts."""
    print(f"Transferring ${amount} from {from_account} to {to_account}")
    return True


@tool_permission(ToolPermission.WRITE, ToolPermission.CONNECT)
def send_payment_notification(email: str, amount: float) -> bool:
    """Send payment notification to external email."""
    print(f"Sending notification to {email} for ${amount}")
    return True


ALL_TOOLS = [get_balance, get_transactions, transfer_funds, send_payment_notification]

In [3]:
for tool in ALL_TOOLS:
    print(f"{tool.__name__}: {get_permissions(tool)}")

get_balance: {<ToolPermission.READ: 'read'>}
get_transactions: {<ToolPermission.READ: 'read'>}
transfer_funds: {<ToolPermission.WRITE: 'write'>}
send_payment_notification: {<ToolPermission.CONNECT: 'connect'>, <ToolPermission.WRITE: 'write'>}


## Construction-Time Filtering

Filter tools before passing them to the agent. The agent only sees tools it has permission to use.

### Read-only agent

In [4]:
read_only_tools = filter_tools_by_permission(ALL_TOOLS, granted={ToolPermission.READ})
print(f"Tools available: {[t.__name__ for t in read_only_tools]}")

Tools available: ['get_balance', 'get_transactions']


In [5]:
agent = get_agent(tools=read_only_tools)
result, _ = await run_agent(
    agent, "What is the balance of account ACC-123?", verbose=True
)
print(f"\nAnswer: {result.result.output}")

Agent step: UserPromptNode(user_prompt='What is the balance of account ACC-123?', instructions_functions=[], 
system_prompts=(), system_prompt_functions=[], system_prompt_dynamic_functions={})

Agent step: ModelRequestNode(request=ModelRequest(parts=[UserPromptPart(content='What is the balance of account 
ACC-123?', timestamp=datetime.datetime(2026, 2, 12, 13, 53, 25, 335353, tzinfo=datetime.timezone.utc))]))

Agent step: CallToolsNode(model_response=ModelResponse(parts=[ToolCallPart(tool_name='get_balance', 
args='{"account_id": "ACC-123"}', tool_call_id='toolu_vrtx_01RCAr4PeVdiZcURFzTf48Xx')], 
usage=RequestUsage(input_tokens=660, output_tokens=57, details={'is_byok': False, 'reasoning_tokens': 0}), 
model_name='anthropic/claude-sonnet-4.5', timestamp=datetime.datetime(2026, 2, 12, 13, 53, 27, 50555, 
tzinfo=datetime.timezone.utc), provider_name='openai', provider_url='https://openrouter.ai/api/v1/', 
provider_details={'finish_reason': 'tool_calls', 'timestamp': datetime.datetime(2026, 2, 12, 13, 53, 25, 
tzinfo=TzInfo(0))}, provider_response_id='gen-1770904405-manu2BN29xC1rktCApry', finish_reason='tool_call', 
run_id='ad044a84-7b88-4ed6-86e6-9041919bed71'))

Reading balance for account: ACC-123


Agent step: ModelRequestNode(request=ModelRequest(parts=[ToolReturnPart(tool_name='get_balance', content=1500.0, 
tool_call_id='toolu_vrtx_01RCAr4PeVdiZcURFzTf48Xx', timestamp=datetime.datetime(2026, 2, 12, 13, 53, 27, 67443, 
tzinfo=datetime.timezone.utc))]))

Agent step: CallToolsNode(model_response=ModelResponse(parts=[TextPart(content='The balance of account ACC-123 is 
**$1,500.00**.')], usage=RequestUsage(input_tokens=734, output_tokens=20, details={'is_byok': False, 
'reasoning_tokens': 0}), model_name='anthropic/claude-sonnet-4.5', timestamp=datetime.datetime(2026, 2, 12, 13, 53,
28, 475435, tzinfo=datetime.timezone.utc), provider_name='openai', provider_url='https://openrouter.ai/api/v1/', 
provider_details={'finish_reason': 'stop', 'timestamp': datetime.datetime(2026, 2, 12, 13, 53, 27, 
tzinfo=TzInfo(0))}, provider_response_id='gen-1770904407-4a8S6R5syLOcL3FRKiTx', finish_reason='stop', 
run_id='ad044a84-7b88-4ed6-86e6-9041919bed71'))

Agent step: End(data=FinalResult(output='The balance of account ACC-123 is **$1,500.00**.'))


Answer: The balance of account ACC-123 is **$1,500.00**.


If asked to transfer funds, the agent cannot comply because it has no access to the transfer tool:

In [6]:
result, _ = await run_agent(
    agent, "Transfer $100 from ACC-123 to ACC-456", verbose=True
)
print(f"\nAnswer: {result.result.output}")

Agent step: UserPromptNode(user_prompt='Transfer $100 from ACC-123 to ACC-456', instructions_functions=[], 
system_prompts=(), system_prompt_functions=[], system_prompt_dynamic_functions={})

Agent step: ModelRequestNode(request=ModelRequest(parts=[UserPromptPart(content='Transfer $100 from ACC-123 to 
ACC-456', timestamp=datetime.datetime(2026, 2, 12, 13, 53, 28, 495506, tzinfo=datetime.timezone.utc))]))

Agent step: CallToolsNode(model_response=ModelResponse(parts=[TextPart(content="I don't have access to a transfer 
function to move money between accounts. The available tools I have are:\n\n1. **get_balance** - to check account 
balances\n2. **get_transactions** - to view recent transactions\n\nTo transfer $100 from ACC-123 to ACC-456, you 
would need to:\n- Use your bank's official transfer feature (through their app, website, or by contacting them 
directly)\n- Or contact your financial institution to process the transfer\n\nWould you like me to check the 
current balance of either account before you proceed with the transfer through your bank?")], 
usage=RequestUsage(input_tokens=663, output_tokens=130, details={'is_byok': False, 'reasoning_tokens': 0}), 
model_name='anthropic/claude-sonnet-4.5', timestamp=datetime.datetime(2026, 2, 12, 13, 53, 31, 598329, 
tzinfo=datetime.timezone.utc), provider_name='openai', provider_url='https://openrouter.ai/api/v1/', 
provider_details={'finish_reason': 'stop', 'timestamp': datetime.datetime(2026, 2, 12, 13, 53, 28, 
tzinfo=TzInfo(0))}, provider_response_id='gen-1770904408-4sXTUmZhw7W6WzkWf6s2', finish_reason='stop', 
run_id='50b53095-0469-4fd7-a4fd-37da6c8f59ea'))

Agent step: End(data=FinalResult(output="I don't have access to a transfer function to move money between accounts.
The available tools I have are:\n\n1. **get_balance** - to check account balances\n2. **get_transactions** - to 
view recent transactions\n\nTo transfer $100 from ACC-123 to ACC-456, you would need to:\n- Use your bank's 
official transfer feature (through their app, website, or by contacting them directly)\n- Or contact your financial
institution to process the transfer\n\nWould you like me to check the current balance of either account before you 
proceed with the transfer through your bank?"))


Answer: I don't have access to a transfer function to move money between accounts. The available tools I have are:

1. **get_balance** - to check account balances
2. **get_transactions** - to view recent transactions

To transfer $100 from ACC-123 to ACC-456, you would need to:
- Use your bank's official transfer feature (through their app, website, or by contacting them directly)
- Or contact your financial institution to process the transfer

Would you like me to check the current balance of either account before you proceed with the transfer through your bank?


### Read-write agent

In [7]:
write_tools = filter_tools_by_permission(
    ALL_TOOLS, granted={ToolPermission.READ, ToolPermission.WRITE}
)
print(f"Tools available: {[t.__name__ for t in write_tools]}")

Tools available: ['get_balance', 'get_transactions', 'transfer_funds']


In [8]:
agent = get_agent(tools=write_tools)
result, _ = await run_agent(
    agent, "Transfer $100 from ACC-123 to ACC-456", verbose=True
)
print(f"\nAnswer: {result.result.output}")

Agent step: UserPromptNode(user_prompt='Transfer $100 from ACC-123 to ACC-456', instructions_functions=[], 
system_prompts=(), system_prompt_functions=[], system_prompt_dynamic_functions={})

Agent step: ModelRequestNode(request=ModelRequest(parts=[UserPromptPart(content='Transfer $100 from ACC-123 to 
ACC-456', timestamp=datetime.datetime(2026, 2, 12, 13, 53, 31, 627634, tzinfo=datetime.timezone.utc))]))

Agent step: CallToolsNode(model_response=ModelResponse(parts=[ToolCallPart(tool_name='transfer_funds', 
args='{"from_account": "ACC-123", "to_account": "ACC-456", "amount": 100}', 
tool_call_id='toolu_vrtx_01RGMzgeyZnmmQz69ZJDYdQp')], usage=RequestUsage(input_tokens=756, output_tokens=95, 
details={'is_byok': False, 'reasoning_tokens': 0}), model_name='anthropic/claude-sonnet-4.5', 
timestamp=datetime.datetime(2026, 2, 12, 13, 53, 33, 147645, tzinfo=datetime.timezone.utc), provider_name='openai',
provider_url='https://openrouter.ai/api/v1/', provider_details={'finish_reason': 'tool_calls', 'timestamp': 
datetime.datetime(2026, 2, 12, 13, 53, 31, tzinfo=TzInfo(0))}, 
provider_response_id='gen-1770904411-ePUnU0sLzZg8YanTIvU4', finish_reason='tool_call', 
run_id='f40e2733-6327-4c37-bcde-b42161527b1a'))

Transferring $100.0 from ACC-123 to ACC-456


Agent step: ModelRequestNode(request=ModelRequest(parts=[ToolReturnPart(tool_name='transfer_funds', content=True, 
tool_call_id='toolu_vrtx_01RGMzgeyZnmmQz69ZJDYdQp', timestamp=datetime.datetime(2026, 2, 12, 13, 53, 33, 151811, 
tzinfo=datetime.timezone.utc))]))

Agent step: CallToolsNode(model_response=ModelResponse(parts=[TextPart(content='The transfer has been completed 
successfully. $100 has been transferred from account ACC-123 to account ACC-456.')], 
usage=RequestUsage(input_tokens=865, output_tokens=28, details={'is_byok': False, 'reasoning_tokens': 0}), 
model_name='anthropic/claude-sonnet-4.5', timestamp=datetime.datetime(2026, 2, 12, 13, 53, 34, 415761, 
tzinfo=datetime.timezone.utc), provider_name='openai', provider_url='https://openrouter.ai/api/v1/', 
provider_details={'finish_reason': 'stop', 'timestamp': datetime.datetime(2026, 2, 12, 13, 53, 33, 
tzinfo=TzInfo(0))}, provider_response_id='gen-1770904413-JMdN7eLAbQGqrhBTcXpK', finish_reason='stop', 
run_id='f40e2733-6327-4c37-bcde-b42161527b1a'))

Agent step: End(data=FinalResult(output='The transfer has been completed successfully. $100 has been transferred 
from account ACC-123 to account ACC-456.'))


Answer: The transfer has been completed successfully. $100 has been transferred from account ACC-123 to account ACC-456.


### Full access agent

In [9]:
full_tools = filter_tools_by_permission(
    ALL_TOOLS,
    granted={ToolPermission.READ, ToolPermission.WRITE, ToolPermission.CONNECT},
)
print(f"Tools available: {[t.__name__ for t in full_tools]}")

Tools available: ['get_balance', 'get_transactions', 'transfer_funds', 'send_payment_notification']


In [10]:
agent = get_agent(tools=full_tools)
result, _ = await run_agent(
    agent,
    "Transfer $100 from ACC-123 to ACC-456 and notify user@example.com",
    verbose=True,
)
print(f"\nAnswer: {result.result.output}")

Agent step: UserPromptNode(user_prompt='Transfer $100 from ACC-123 to ACC-456 and notify user@example.com', 
instructions_functions=[], system_prompts=(), system_prompt_functions=[], system_prompt_dynamic_functions={})

Agent step: ModelRequestNode(request=ModelRequest(parts=[UserPromptPart(content='Transfer $100 from ACC-123 to 
ACC-456 and notify user@example.com', timestamp=datetime.datetime(2026, 2, 12, 13, 53, 34, 452062, 
tzinfo=datetime.timezone.utc))]))

Agent step: CallToolsNode(model_response=ModelResponse(parts=[TextPart(content="I'll transfer $100 from ACC-123 to 
ACC-456 and send a payment notification to user@example.com."), ToolCallPart(tool_name='transfer_funds', 
args='{"from_account": "ACC-123", "to_account": "ACC-456", "amount": 100}', 
tool_call_id='toolu_vrtx_01RwnCG1k1KDSY3TdWYqHEnR'), ToolCallPart(tool_name='send_payment_notification', 
args='{"email": "user@example.com", "amount": 100}', tool_call_id='toolu_vrtx_01AnV35ZBLBvyka9gWvAHGUc')], 
usage=RequestUsage(input_tokens=840, output_tokens=183, details={'is_byok': False, 'reasoning_tokens': 0}), 
model_name='anthropic/claude-sonnet-4.5', timestamp=datetime.datetime(2026, 2, 12, 13, 53, 36, 668532, 
tzinfo=datetime.timezone.utc), provider_name='openai', provider_url='https://openrouter.ai/api/v1/', 
provider_details={'finish_reason': 'tool_calls', 'timestamp': datetime.datetime(2026, 2, 12, 13, 53, 34, 
tzinfo=TzInfo(0))}, provider_response_id='gen-1770904414-ltq6dRfAUs1lxTutIJ1e', finish_reason='tool_call', 
run_id='bcb8d20c-162d-4a58-874c-886b8d67680b'))

Transferring $100.0 from ACC-123 to ACC-456
Sending notification to user@example.com for $100.0


Agent step: ModelRequestNode(request=ModelRequest(parts=[ToolReturnPart(tool_name='transfer_funds', content=True, 
tool_call_id='toolu_vrtx_01RwnCG1k1KDSY3TdWYqHEnR', timestamp=datetime.datetime(2026, 2, 12, 13, 53, 36, 672565, 
tzinfo=datetime.timezone.utc)), ToolReturnPart(tool_name='send_payment_notification', content=True, 
tool_call_id='toolu_vrtx_01AnV35ZBLBvyka9gWvAHGUc', timestamp=datetime.datetime(2026, 2, 12, 13, 53, 36, 672954, 
tzinfo=datetime.timezone.utc))]))

Agent step: CallToolsNode(model_response=ModelResponse(parts=[TextPart(content="Perfect! I've successfully 
completed both tasks:\n\n1. ✅ Transferred $100 from ACC-123 to ACC-456\n2. ✅ Sent a payment notification to 
user@example.com for the $100 amount\n\nBoth operations completed successfully.")], 
usage=RequestUsage(input_tokens=1090, output_tokens=63, details={'is_byok': False, 'reasoning_tokens': 0}), 
model_name='anthropic/claude-sonnet-4.5', timestamp=datetime.datetime(2026, 2, 12, 13, 53, 38, 234103, 
tzinfo=datetime.timezone.utc), provider_name='openai', provider_url='https://openrouter.ai/api/v1/', 
provider_details={'finish_reason': 'stop', 'timestamp': datetime.datetime(2026, 2, 12, 13, 53, 36, 
tzinfo=TzInfo(0))}, provider_response_id='gen-1770904416-mesJPGJ5BpfSEstpx0Xl', finish_reason='stop', 
run_id='bcb8d20c-162d-4a58-874c-886b8d67680b'))

Agent step: End(data=FinalResult(output="Perfect! I've successfully completed both tasks:\n\n1. ✅ Transferred $100
from ACC-123 to ACC-456\n2. ✅ Sent a payment notification to user@example.com for the $100 amount\n\nBoth 
operations completed successfully."))


Answer: Perfect! I've successfully completed both tasks:

1. ✅ Transferred $100 from ACC-123 to ACC-456
2. ✅ Sent a payment notification to user@example.com for the $100 amount

Both operations completed successfully.


## Runtime Enforcement

An alternative approach: the agent sees all tools, but execution fails if permission is denied. Use `enforce_tools_permissions` to wrap tools with permission checks baked in.

### Agent with all tools visible, read-only enforcement

In [11]:
enforced_tools = enforce_tools_permissions(ALL_TOOLS, granted={ToolPermission.READ})
print(f"Tools visible to agent: {[t.__name__ for t in enforced_tools]}")

Tools visible to agent: ['get_balance', 'get_transactions', 'transfer_funds', 'send_payment_notification']


In [12]:
agent = get_agent(tools=enforced_tools)
result, _ = await run_agent(
    agent, "What is the balance of account ACC-123?", verbose=True
)
print(f"\nAnswer: {result.result.output}")

Agent step: UserPromptNode(user_prompt='What is the balance of account ACC-123?', instructions_functions=[], 
system_prompts=(), system_prompt_functions=[], system_prompt_dynamic_functions={})

Agent step: ModelRequestNode(request=ModelRequest(parts=[UserPromptPart(content='What is the balance of account 
ACC-123?', timestamp=datetime.datetime(2026, 2, 12, 13, 53, 38, 264042, tzinfo=datetime.timezone.utc))]))

Agent step: CallToolsNode(model_response=ModelResponse(parts=[ToolCallPart(tool_name='get_balance', 
args='{"account_id": "ACC-123"}', tool_call_id='toolu_vrtx_01WPMqLAC3g9n6HnWAzHwjNw')], 
usage=RequestUsage(input_tokens=829, output_tokens=57, details={'is_byok': False, 'reasoning_tokens': 0}), 
model_name='anthropic/claude-sonnet-4.5', timestamp=datetime.datetime(2026, 2, 12, 13, 53, 39, 611337, 
tzinfo=datetime.timezone.utc), provider_name='openai', provider_url='https://openrouter.ai/api/v1/', 
provider_details={'finish_reason': 'tool_calls', 'timestamp': datetime.datetime(2026, 2, 12, 13, 53, 38, 
tzinfo=TzInfo(0))}, provider_response_id='gen-1770904418-PGh1yJbGv2YgnwTmjhJA', finish_reason='tool_call', 
run_id='21cf986d-21f4-45f4-af01-1f663dbe8163'))

Reading balance for account: ACC-123


Agent step: ModelRequestNode(request=ModelRequest(parts=[ToolReturnPart(tool_name='get_balance', content=1500.0, 
tool_call_id='toolu_vrtx_01WPMqLAC3g9n6HnWAzHwjNw', timestamp=datetime.datetime(2026, 2, 12, 13, 53, 39, 615886, 
tzinfo=datetime.timezone.utc))]))

Agent step: CallToolsNode(model_response=ModelResponse(parts=[TextPart(content='The balance of account ACC-123 is 
**$1,500.00**.')], usage=RequestUsage(input_tokens=903, output_tokens=20, details={'is_byok': False, 
'reasoning_tokens': 0}), model_name='anthropic/claude-sonnet-4.5', timestamp=datetime.datetime(2026, 2, 12, 13, 53,
41, 469591, tzinfo=datetime.timezone.utc), provider_name='openai', provider_url='https://openrouter.ai/api/v1/', 
provider_details={'finish_reason': 'stop', 'timestamp': datetime.datetime(2026, 2, 12, 13, 53, 39, 
tzinfo=TzInfo(0))}, provider_response_id='gen-1770904419-OcQeucHQ1y4dgEGbaszP', finish_reason='stop', 
run_id='21cf986d-21f4-45f4-af01-1f663dbe8163'))

Agent step: End(data=FinalResult(output='The balance of account ACC-123 is **$1,500.00**.'))


Answer: The balance of account ACC-123 is **$1,500.00**.


The agent sees transfer_funds and may try to use it, but execution raises ToolPermissionError:

In [15]:
from agentic_patterns.core.tools.permissions import ToolPermissionError


try:
    result, _ = await run_agent(agent, "Transfer $100 from ACC-123 to ACC-456", verbose=True)
    print(f"\nAnswer: {result.result.output}")
except ToolPermissionError as e:
    print(f"ERROR: Tool not allowed: {e}")

Agent step: UserPromptNode(user_prompt='Transfer $100 from ACC-123 to ACC-456', instructions_functions=[], 
system_prompts=(), system_prompt_functions=[], system_prompt_dynamic_functions={})

Agent step: ModelRequestNode(request=ModelRequest(parts=[UserPromptPart(content='Transfer $100 from ACC-123 to 
ACC-456', timestamp=datetime.datetime(2026, 2, 12, 13, 55, 43, 154768, tzinfo=datetime.timezone.utc))]))

Agent step: CallToolsNode(model_response=ModelResponse(parts=[ToolCallPart(tool_name='transfer_funds', 
args='{"from_account": "ACC-123", "to_account": "ACC-456", "amount": 100}', 
tool_call_id='toolu_vrtx_01Gsszd52TGvbkji5otnemBJ')], usage=RequestUsage(input_tokens=832, output_tokens=95, 
details={'is_byok': False, 'reasoning_tokens': 0}), model_name='anthropic/claude-sonnet-4.5', 
timestamp=datetime.datetime(2026, 2, 12, 13, 55, 45, 833316, tzinfo=datetime.timezone.utc), provider_name='openai',
provider_url='https://openrouter.ai/api/v1/', provider_details={'finish_reason': 'tool_calls', 'timestamp': 
datetime.datetime(2026, 2, 12, 13, 55, 44, tzinfo=TzInfo(0))}, 
provider_response_id='gen-1770904544-g8Y9NzaVBxjeCzClFNai', finish_reason='tool_call', 
run_id='c5d22a92-1b52-4480-bad7-a318f058af76'))

Error running agent: Tool 'transfer_funds' requires {<ToolPermission.WRITE: 'write'>}

ERROR: Tool not allowed: Tool 'transfer_funds' requires {<ToolPermission.WRITE: 'write'>}


### Agent with write enforcement

In [16]:
enforced_tools = enforce_tools_permissions(
    ALL_TOOLS, granted={ToolPermission.READ, ToolPermission.WRITE}
)
agent = get_agent(tools=enforced_tools)
result, _ = await run_agent(
    agent, "Transfer $100 from ACC-123 to ACC-456", verbose=True
)
print(f"\nAnswer: {result.result.output}")

Agent step: UserPromptNode(user_prompt='Transfer $100 from ACC-123 to ACC-456', instructions_functions=[], 
system_prompts=(), system_prompt_functions=[], system_prompt_dynamic_functions={})

Agent step: ModelRequestNode(request=ModelRequest(parts=[UserPromptPart(content='Transfer $100 from ACC-123 to 
ACC-456', timestamp=datetime.datetime(2026, 2, 12, 13, 55, 46, 872488, tzinfo=datetime.timezone.utc))]))

Agent step: CallToolsNode(model_response=ModelResponse(parts=[ToolCallPart(tool_name='transfer_funds', 
args='{"from_account": "ACC-123", "to_account": "ACC-456", "amount": 100}', 
tool_call_id='toolu_vrtx_012sTP9BaBHD2w97gxxaqyKJ')], usage=RequestUsage(input_tokens=832, output_tokens=95, 
details={'is_byok': False, 'reasoning_tokens': 0}), model_name='anthropic/claude-sonnet-4.5', 
timestamp=datetime.datetime(2026, 2, 12, 13, 55, 48, 875921, tzinfo=datetime.timezone.utc), provider_name='openai',
provider_url='https://openrouter.ai/api/v1/', provider_details={'finish_reason': 'tool_calls', 'timestamp': 
datetime.datetime(2026, 2, 12, 13, 55, 46, tzinfo=TzInfo(0))}, 
provider_response_id='gen-1770904546-tk2e3CdJ02eA0jaQ7JWq', finish_reason='tool_call', 
run_id='e246002b-8d13-4f7b-82d8-bdb9ac62e902'))

Transferring $100.0 from ACC-123 to ACC-456


Agent step: ModelRequestNode(request=ModelRequest(parts=[ToolReturnPart(tool_name='transfer_funds', content=True, 
tool_call_id='toolu_vrtx_012sTP9BaBHD2w97gxxaqyKJ', timestamp=datetime.datetime(2026, 2, 12, 13, 55, 48, 883917, 
tzinfo=datetime.timezone.utc))]))

Agent step: CallToolsNode(model_response=ModelResponse(parts=[TextPart(content='The transfer of $100 from ACC-123 
to ACC-456 has been completed successfully.')], usage=RequestUsage(input_tokens=941, output_tokens=24, 
details={'is_byok': False, 'reasoning_tokens': 0}), model_name='anthropic/claude-sonnet-4.5', 
timestamp=datetime.datetime(2026, 2, 12, 13, 55, 49, 986564, tzinfo=datetime.timezone.utc), provider_name='openai',
provider_url='https://openrouter.ai/api/v1/', provider_details={'finish_reason': 'stop', 'timestamp': 
datetime.datetime(2026, 2, 12, 13, 55, 48, tzinfo=TzInfo(0))}, 
provider_response_id='gen-1770904548-ZmkWRCm9G40zx5w9mKqt', finish_reason='stop', 
run_id='e246002b-8d13-4f7b-82d8-bdb9ac62e902'))

Agent step: End(data=FinalResult(output='The transfer of $100 from ACC-123 to ACC-456 has been completed 
successfully.'))


Answer: The transfer of $100 from ACC-123 to ACC-456 has been completed successfully.


## Comparison

**Construction-time filtering** (`filter_tools_by_permission`): The agent never sees disallowed tools. Cleaner, but the agent cannot explain why a capability is unavailable.

**Runtime enforcement** (`enforce_tools_permissions`): The agent sees all tools but execution fails if denied. The agent can attempt the action and receive an error, potentially explaining the limitation to the user.